In [ ]:
import json
import pickle
import numpy as np
from PIL import Image
from pynq import Overlay
import pynq.lib.dma
from pynq import allocate
import sys

sys.path.append("./pynq")
import IPUnit_PYNQ

In [ ]:
json_file = "./pynq/Config.json"
with open(json_file, "r") as jf:
    ip_config = json.load(jf)

# data_types_dict = {"data_type": "q4.12",
#                    "addr_type": "int16",
#                    "signal_type": "int16"}
_data_types_dict = ip_config["data_types"]
data_types_dict = {}
for k, v in _data_types_dict.items():
    if "_t" in v:
        data_types_dict[k] = v[:-2]
    elif v == "float":
        data_types_dict[k] = "float32"
    elif v == "double":
        data_types_dict[k] = "float64"
    else:
        data_types_dict[k] = v
print(data_types_dict)

data_types_dataset = np.float32
src_size, dst_size = (1, 28, 28, 1), (1, 28, 28, 1)

fake_data_src = np.array(Image.open('./keras_mnist_DCAE_input_2D.png'), dtype=data_types_dataset)
fake_data_src = (fake_data_src - np.min(fake_data_src)) / (np.max(fake_data_src) - np.min(fake_data_src))
fake_data_src = [fake_data_src.reshape(src_size)]

data_vec_config = ip_config["data_vec_config"]
_src_data = [[0 for x in range(y)] for y in data_vec_config.values()]
for bank, i in enumerate(fake_data_src):
    _i = i.reshape(-1).tolist()
    for j, k in enumerate(_i):
        _src_data[bank][j] = k
opcode = ip_config["opcode"]

control_code_file = "./pynq/control_code.bin"
weights_vec_file = "./pynq/Weights.bin"
bitstream_file = "./pynq/design_1.bit"
ipcore_name = "ipcore_7threads_q4_12_0"

ipunit_output = IPUnit_PYNQ.IPUnit_PYNQ(_src_data,
                                        opcode,
                                        control_code_file=control_code_file,
                                        weights_vec_file=weights_vec_file,
                                        bitstream_file=bitstream_file,
                                        data_types=data_types_dict,
                                        ipcore_name=ipcore_name,
                                        print_debug=True)

In [ ]:
def export_as_png(data: list, shape, file_name: str):
    _output = np.array(data).reshape(shape)
    _output = (_output - np.min(_output)) / (np.max(_output) - np.min(_output)) * 255
    _output = _output.astype(np.uint8)
    _img = Image.fromarray(_output)
    _img.save(file_name)

_dst_size = dst_size[0] * dst_size[1] * dst_size[2] * dst_size[3]
ipunit_output[0] = ipunit_output[0][0:_dst_size]

export_as_png(ipunit_output[0], dst_size[1:3], "./result_img/ipunit_output_pynq.png")
Image.open("./result_img/ipunit_output_pynq.png")

In [ ]:
output_bin_files = ["./result_img/pynq_output.bin"]
output_bin_data = [ipunit_output[0]]
for serialized_item in zip(output_bin_files, output_bin_data):
    with open(serialized_item[0], "wb") as f:
        pickle.dump(serialized_item[1], f)

In [ ]:
src_data_name = ["./test_img/src_data_{:03}.png".format(i) for i in range(100)]
execute_time = []

for cnt, src_img in enumerate(src_data_name):
    fake_data_src = np.array(Image.open(src_img), dtype=data_types_dataset)
    fake_data_src = (fake_data_src - np.min(fake_data_src)) / (np.max(fake_data_src) - np.min(fake_data_src))
    fake_data_src = [fake_data_src.reshape(src_size)]

    data_vec_config = ip_config["data_vec_config"]
    _src_data = [[0 for x in range(y)] for y in data_vec_config.values()]
    for bank, i in enumerate(fake_data_src):
        _i = i.reshape(-1).tolist()
        for j, k in enumerate(_i):
            _src_data[bank][j] = k
    opcode = ip_config["opcode"]

    ipunit_output, _execute_time = IPUnit_PYNQ.IPUnit_PYNQ_execute_time(_src_data,
                                                                       opcode,
                                                                       control_code_file=control_code_file,
                                                                       weights_vec_file=weights_vec_file,
                                                                       bitstream_file=bitstream_file,
                                                                       data_types=data_types_dict,
                                                                       ipcore_name=ipcore_name,
                                                                       print_debug=False)
    execute_time.append(_execute_time)


In [ ]:
run_time = []
for i, single_record in enumerate(execute_time):
    for j, kv in enumerate(single_record.items()):
        k, v = kv
        if j == 0:
            fields = list(v.keys())
            single_run_time = [0.0 for x in range(len(v))]
        _values = list(v.values())
        print(_values)
        for k, _time in enumerate(_values):
            single_run_time[k] += _time
    run_time.append(single_run_time)

print()
print(fields)
print(run_time)
print("total_time(average) : {}[ms]".format(sum(map(sum, run_time)) / len(run_time)))
with open("./result_img/result_time_pynq.time", "wb") as f:
    pickle.dump({"field": fields, "run_time": run_time, "execute_time": execute_time, "device": "pynqZ2"}, f)

In [ ]:
run_core = []
for i in run_time:
    run_core.append(i[-2])
print("run_core_average : {} \pm {}[ms]".format(np.mean(run_core), np.std(run_core)))